Pruebas PSN

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup as bs
import requests


In [27]:
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

Prueba primera con google

In [52]:
driver = webdriver.Chrome(service=service, options=options)

driver.get("http://www.google.es")

time.sleep(3)

rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/span/div/div/div/div[3]/div[1]/button[1]/div')
rechazar_cookies.click()

time.sleep(3)

driver.quit()

Haciendo pruebas con psn

In [50]:
df_juegos = pd.DataFrame(columns=["Titulo","Precio_original","Precio_actual","Precio con mayor rebaja"])

In [51]:
df_juegos

,Titulo,Precio_original,Precio_actual,Precio con mayor rebaja


In [ ]:


driver = webdriver.Chrome(service=service, options=options)

driver.get("https://store.playstation.com/")


lista_tiempo = [1.5, 2, 2.5, 3.5, 4, 5, 6.5, 7]

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()

#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

#seleccion de juego

time.sleep(random.choice(lista_tiempo))
select_game = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[1]/div/a/div/div/div[1]/span[2]')
select_game.click()

time.sleep(random.choice(lista_tiempo))
#Info que nos descargamos de la pagina del juego.
page_source = driver.page_source
soup = bs(page_source, 'lxml')
# df_juegos = df_juegos.append({"Titulo",soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()})
# df_juegos = df_juegos.append({"Precio_original" ,soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()})
# df_juegos = df_juegos.append({"Precio_actual" ,soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()})
time.sleep(random.choice(lista_tiempo))

driver.back();

In [70]:
df_juegos

,Titulo,Precio_original,Precio_actual,Precio con mayor rebaja


In [61]:
#Titulo
soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()

'01 Deadliest Zone Catch — Boat Crab & Fishing Simulator'

In [62]:
#Precio Original 
soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()


'14,99\xa0€'

In [63]:
#Precio Actual
soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()

'8,99\xa0€'